<a href="https://colab.research.google.com/github/Hussain1488/Fake-News-Detection/blob/main/FakeNewsDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install torchtext kagglehub
# !pip uninstall -y torch torchtext
!pip install torch==2.3.0 torchtext==0.18.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [19]:
import kagglehub
import torch
import torchtext
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from torchtext.data.utils import get_tokenizer
from nltk.stem import PorterStemmer
from torch import nn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
path = kagglehub.dataset_download("clmentbisaillon/fake-and-real-news-dataset")
print("Path to dataset files:", path)


Path to dataset files: /kaggle/input/fake-and-real-news-dataset


# Preprocessing:

In [21]:

fake=pd.read_csv(path + '/Fake.csv')
real=pd.read_csv(path + '/True.csv')
fake['label'] = 0
real['label'] = 1


class preprocessing:

  def __init__(self, fake, real):
    self.fake = fake
    self.real = real

  def stop_word_removal(self):

    stop_words = set(stopwords.words('english'))

    for i, item in self.fake.iterrows():
      filtered_text = [token.lower() for token in item['text'].split() if token.lower() not in stop_words]
      filtered_title = [token.lower() for token in item['title'].split() if token.lower() not in stop_words]
      fake.at[i, 'text'] = filtered_text
      fake.at[i, 'title'] = filtered_title

    for i, item in self.real.iterrows():
      filtered_text = [token.lower() for token in item['text'].split() if token.lower() not in stop_words]
      filtered_title = [token.lower() for token in item['title'].split() if token.lower() not in stop_words]
      real.at[i, 'text'] = filtered_text
      real.at[i, 'title'] = filtered_title
      df = pd.concat([fake, real])
    return df

  def stemming(self, dataset):
    stemmer = PorterStemmer()
    dataset['text'] = dataset['text'].astype(object)
    dataset['title'] = dataset['title'].astype(object)

    dataset['text'] = dataset['text'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
    dataset['title'] = dataset['title'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
    return dataset

  def title_one_hot(self, dataset):
    one_hot = pd.get_dummies(dataset['subject'], prefix='subject', dtype=int, drop_first=True)
    dataset = pd.concat([dataset, one_hot], axis=1)
    return dataset

  def main(self):
    df = self.stop_word_removal()
    df = self.stemming(df)
    df = self.title_one_hot(df)
    return df


preprocessing = preprocessing(fake, real)
df = preprocessing.main()



df

,title,text,subject,date,label,subject_Middle-east,subject_News,subject_US_News,subject_left-news,subject_politics,subject_politicsNews,subject_worldnews
0,"[donald, trump, send, embarrass, new, year’, e...","[donald, trump, wish, american, happi, new, ye...",News,"December 31, 2017",0,0,1,0,0,0,0,0
1,"[drunk, brag, trump, staffer, start, russian, ...","[hous, intellig, committe, chairman, devin, nu...",News,"December 31, 2017",0,0,1,0,0,0,0,0
2,"[sheriff, david, clark, becom, internet, joke,...","[friday,, reveal, former, milwauke, sheriff, d...",News,"December 30, 2017",0,0,1,0,0,0,0,0
3,"[trump, obsess, even, obama’, name, code, webs...","[christma, day,, donald, trump, announc, would...",News,"December 29, 2017",0,0,1,0,0,0,0,0
4,"[pope, franci, call, donald, trump, christma, ...","[pope, franci, use, annual, christma, day, mes...",News,"December 25, 2017",0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
21412,"['fulli, committed', nato, back, new, u.s., ap...","[brussel, (reuters), -, nato, alli, tuesday, w...",worldnews,"August 22, 2017",1,0,0,0,0,0,0,1
21413,"[lexisnexi, withdrew, two, product, chines, ma...","[london, (reuters), -, lexisnexis,, provid, le...",worldnews,"August 22, 2017",1,0,0,0,0,0,0,1
21414,"[minsk, cultur, hub, becom, author]","[minsk, (reuters), -, shadow, disus, soviet-er...",worldnews,"August 22, 2017",1,0,0,0,0,0,0,1
21415,"[vatican, upbeat, possibl, pope, franci, visit...","[moscow, (reuters), -, vatican, secretari, sta...",worldnews,"August 22, 2017",1,0,0,0,0,0,0,1


In [22]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 44898 entries, 0 to 21416
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   title                 44898 non-null  object
 1   text                  44898 non-null  object
 2   subject               44898 non-null  object
 3   date                  44898 non-null  object
 4   label                 44898 non-null  int64 
 5   subject_Middle-east   44898 non-null  int64 
 6   subject_News          44898 non-null  int64 
 7   subject_US_News       44898 non-null  int64 
 8   subject_left-news     44898 non-null  int64 
 9   subject_politics      44898 non-null  int64 
 10  subject_politicsNews  44898 non-null  int64 
 11  subject_worldnews     44898 non-null  int64 
dtypes: int64(8), object(4)
memory usage: 4.5+ MB


In [16]:
df['clean_text'] = df['text'].apply(lambda x: " ".join(x))

tfidf = TfidfVectorizer(max_features=5000)  # you can tune max_features
X = tfidf.fit_transform(df['clean_text'])
y = df['label']

# ====================
# Train / Test Split
# ====================
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

Train shape: (35918, 5000)
Test shape: (8980, 5000)


# Vectorization

In [24]:
X_train_tensor = torch.tensor(X_train.toarray(), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.toarray(), dtype=torch.float32)

y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Input dimension
input_dim = X_train_tensor.shape[1]
def train_and_eval(lr=0.001, batch_size=64, hidden1=512, hidden2=256, dropout=0.3, epochs=5):
    # Reload data with new batch_size
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    # Define model
    class FakeNewsClassifier(nn.Module):
        def __init__(self, input_dim):
            super(FakeNewsClassifier, self).__init__()
            self.fc1 = nn.Linear(input_dim, hidden1)
            self.drop1 = nn.Dropout(dropout)
            self.fc2 = nn.Linear(hidden1, hidden2)
            self.drop2 = nn.Dropout(dropout)
            self.fc3 = nn.Linear(hidden2, 1)
            self.sigmoid = nn.Sigmoid()
        def forward(self, x):
            x = torch.relu(self.fc1(x))
            x = self.drop1(x)
            x = torch.relu(self.fc2(x))
            x = self.drop2(x)
            return self.sigmoid(self.fc3(x))

    model = FakeNewsClassifier(input_dim)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Train
    for epoch in range(epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X).squeeze()
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluate
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            outputs = model(batch_X).squeeze()
            preds = (outputs > 0.5).float()
            correct += (preds == batch_y).sum().item()
            total += batch_y.size(0)
    acc = correct / total
    return acc

# Try different settings
results = []
for lr in [0.01, 0.001, 0.0001]:
    for bs in [32, 64, 128]:
        for epoch in [1,2,3,4,5,6,7,8,9,10]:
            acc = train_and_eval(lr=lr, batch_size=bs, epochs=epoch)
            results.append((lr, bs, acc))
            print(f"lr={lr}, batch_size={bs}, epoch={epoch} acc={acc:.4f}")




lr=0.01, batch_size=32, epoch=1 acc=0.9804
lr=0.01, batch_size=32, epoch=2 acc=0.9932
lr=0.01, batch_size=32, epoch=3 acc=0.9929
lr=0.01, batch_size=32, epoch=4 acc=0.9947
lr=0.01, batch_size=32, epoch=5 acc=0.9934
lr=0.01, batch_size=32, epoch=6 acc=0.9945
lr=0.01, batch_size=32, epoch=7 acc=0.9954
lr=0.01, batch_size=32, epoch=8 acc=0.9960
lr=0.01, batch_size=32, epoch=9 acc=0.9960
lr=0.01, batch_size=32, epoch=10 acc=0.9960
lr=0.01, batch_size=64, epoch=1 acc=0.9922
lr=0.01, batch_size=64, epoch=2 acc=0.9929
lr=0.01, batch_size=64, epoch=3 acc=0.9934
lr=0.01, batch_size=64, epoch=4 acc=0.9938
lr=0.01, batch_size=64, epoch=5 acc=0.9912
lr=0.01, batch_size=64, epoch=6 acc=0.9955
lr=0.01, batch_size=64, epoch=7 acc=0.9941
lr=0.01, batch_size=64, epoch=8 acc=0.9931
lr=0.01, batch_size=64, epoch=9 acc=0.9950
lr=0.01, batch_size=64, epoch=10 acc=0.9947
lr=0.01, batch_size=128, epoch=1 acc=0.9909
lr=0.01, batch_size=128, epoch=2 acc=0.9933
lr=0.01, batch_size=128, epoch=3 acc=0.9931
lr=0.0

In [1]:
# Train best model
best_acc = train_and_eval(lr=0.0001, batch_size=32, epochs=10)

# Evaluate again, but now collect predictions
best_model = None

def train_and_return_model(lr=0.0001, batch_size=32, hidden1=512, hidden2=256, dropout=0.3, epochs=10):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    class FakeNewsClassifier(nn.Module):
        def __init__(self, input_dim):
            super().__init__()
            self.fc1 = nn.Linear(input_dim, hidden1)
            self.drop1 = nn.Dropout(dropout)
            self.fc2 = nn.Linear(hidden1, hidden2)
            self.drop2 = nn.Dropout(dropout)
            self.fc3 = nn.Linear(hidden2, 1)
            self.sigmoid = nn.Sigmoid()
        def forward(self, x):
            x = torch.relu(self.fc1(x))
            x = self.drop1(x)
            x = torch.relu(self.fc2(x))
            x = self.drop2(x)
            return self.sigmoid(self.fc3(x))

    model = FakeNewsClassifier(input_dim)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Training
    for epoch in range(epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X).squeeze()
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    return model

# Train and get model
best_model = train_and_return_model(lr=0.0001, batch_size=32, epochs=10)

# Collect predictions
test_loader = DataLoader(test_dataset, batch_size=32)
y_true, y_pred = [], []

best_model.eval()
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        outputs = best_model(batch_X).squeeze()
        preds = (outputs > 0.5).float()
        y_true.extend(batch_y.tolist())
        y_pred.extend(preds.tolist())


NameError: name 'train_and_eval' is not defined

In [ ]:
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Fake","Real"], yticklabels=["Fake","Real"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix (Best Model)")
plt.show()

print(classification_report(y_true, y_pred, target_names=["Fake","Real"]))


In [25]:

all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        outputs = model(batch_X).squeeze()
        preds = (outputs > 0.5).float()
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(batch_y.cpu().numpy())


cm = confusion_matrix(all_labels, all_preds)

plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=["Fake (0)", "Real (1)"],
            yticklabels=["Fake (0)", "Real (1)"])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()


print(classification_report(all_labels, all_preds, target_names=["Fake", "Real"]))


NameError: name 'model' is not defined